In [1]:
# Get the top n number of companies in the Indian, Australian and USA markets
from bs4 import BeautifulSoup
import pandas as pd
import requests
import tkinter as tk
from tkinter import ttk
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Define the number
n=50

def market_companies(url, market, name, symbol, n=1):
    # Send a GET request to the URL and parse the webpage with BeautifulSoup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    if market.upper() == "INDIA":
        # Find all elements with class 'name-td'
        td_elements = soup.find_all('td', class_='name-td')
        
        # Extract company names and codes, up to the specified limit n
        company_data = []
        for td in td_elements[:n]:
            # Find the nested div elements
            name_div = td.find('div', class_='name-div')
            # Extract company name and code from the nested divs
            company_name = name_div.find('div', class_='company-name').text.strip()
            company_code = name_div.find('div', class_='company-code').text.strip()
            
            if company_code.endswith(".BSE"):
                company_code = company_code.replace(".NS", ".BSE")
            elif not company_code.endswith(".NS"):
                company_code += ".NS"
                
            company_data.append({"Market": market.upper(), 'Name': company_name, 'Code': company_code})

            
            df = pd.DataFrame(company_data)
    
    elif market.upper() in ("USA", "AUSTRALIA","UNITED KINGDOM","CANADA"):
        # Find all rows in the table
        rows = soup.find_all('tr')

        # Create lists to store the symbols and names
        symbols = []
        names = []

        # Iterate over each row, up to the specified limit n
        for row in rows[1:n+1]:
            # Find the symbol and name in each row
            namedata = row.find('td', {'class': name})
            symboldata = row.find('td', {'class': symbol})
            
            # If both symbol and name are found, append them to the lists
            if symboldata and namedata:
                symbols.append(symboldata.text.strip())
                names.append(namedata.text.strip())
 
        # Create a DataFrame from the lists
        df = pd.DataFrame({"Market": market.upper(), 'Name': names, 'Code': symbols})

    else:
        raise ValueError("Invalid market value. Please provide one of: 'INDIA', 'USA', 'AUS','TOR', 'LON'.")

    return df

# Example usage
df_India = market_companies("https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/", "INDIA", 'tablebluelink', 'tdcolumn', n)
df_USA = market_companies("https://stockanalysis.com/list/sp-500-stocks/","USA",'slw svelte-eurwtr','sym svelte-eurwtr', n)
df_Aus= market_companies("https://stockanalysis.com/list/australian-securities-exchange/","AUSTRALIA",'slw svelte-eurwtr','sym svelte-eurwtr', n)
df_Lon= market_companies("https://stockanalysis.com/list/london-stock-exchange/","UNITED KINGDOM",'slw svelte-eurwtr','sym svelte-eurwtr', n)
df_Tor = market_companies("https://stockanalysis.com/list/toronto-stock-exchange/","CANADA",'slw svelte-eurwtr','sym svelte-eurwtr', n)


df_companies = pd.concat([df_India, df_Aus, df_USA, df_Lon, df_Tor], ignore_index=True)
df_companies

,Market,Name,Code
0,INDIA,Reliance Industries,RELIANCE.NS
1,INDIA,Tata Consultancy Services,TCS.NS
2,INDIA,HDFC Bank,HDB.NS
3,INDIA,Bharti Airtel,BHARTIARTL.NS
4,INDIA,ICICI Bank,IBN.NS
...,...,...,...
245,CANADA,Ivanhoe Mines Ltd.,IVN
246,CANADA,WSP Global Inc.,WSP
247,CANADA,Power Corporation of Canada,POW
248,CANADA,George Weston Limited,WN


In [ ]:
# Create a GUI with the
# - 2 drop down with a choice of markets and then the top companies from that market
# - Gives the user a button to retrieve share price
# - Shows the share price of the last year for the stock.


class CompanySelectionApp:
    def __init__(self, root, df_companies):
        self.root = root
        self.df_companies = df_companies
        self.root.title("Company Selection")
        self.root.geometry("500x400")

        # List of countries
        self.Countries = list(set(df_companies["Market"].tolist()))

        # Create label for country selection
        self.label_country = ttk.Label(root, text="Select a country:")
        self.label_country.pack(pady=10)

        # Create country combobox
        self.country_combobox = ttk.Combobox(root, values=self.Countries)
        self.country_combobox.pack(pady=5)
        self.country_combobox.bind("<<ComboboxSelected>>", self.get_companies_list)
        self.country_combobox.current(0)


        # Create label for company selection
        self.label_company = ttk.Label(root, text="Select a company:")
        self.label_company.pack(pady=10)

        # Create company combobox
        self.company_combobox = ttk.Combobox(root)
        self.company_combobox.pack(pady=5)

        # Create button to get selected company
        self.get_company_button = ttk.Button(root, text="Get Company Share Price", command=self.get_company)
        self.get_company_button.pack(pady=5)

    # Function to update company combobox based on selected country
    def get_companies_list(self, event=None):
        selected_country = self.country_combobox.get()
        companies = (self.df_companies.loc[self.df_companies["Market"]==selected_country,"Name"].values.tolist())        
        self.company_combobox.config(values=companies)
        self.company_combobox.current(0)

    # Function to get selected company
    def get_company(self):
        selected_company = self.company_combobox.get()
        selected_country = self.country_combobox.get()

        company = self.df_companies.loc[self.df_companies["Name"]==selected_company,"Code"].values.tolist()
        if selected_country == "INDIA":
            company_code = str(company[0])
            currency="INR"
        elif selected_country == "AUSTRALIA":
            company_code = str(company[0])+".AX"
            currency="AUD"
        elif selected_country == "UNITED KINGDOM":
            company_code = company[0]+".L"
            currency="GBP"
        elif selected_country == "CANADA":
            company_code = company[0]+".TO"
            currency="CAD"
        else:
            company_code = str(company[0])
            currency = "USD"

        
        print("Selected company:", selected_company)
        print("Company code:", company_code)

        df = yf.download(tickers=company_code, period='1y', interval='1d')
        self.update_plot(df, company_code,selected_company,currency)
    
#     Function to create a graph with the share price of the stock
    def update_plot(self, df, stock,name,currency):
        # Plot candlestick chart using Matplotlib
        fig, ax = plt.subplots()
        ax.plot(df.index, df['Open'], color='black', label='Open')
        ax.plot(df.index, df['High'], color='green', label='High')
        ax.plot(df.index, df['Low'], color='red', label='Low')
        ax.plot(df.index, df['Close'], color='blue', label='Close')
        ax.set_title(f"{name} Share Price")
        ax.set_ylabel(f"Stock Price ({currency} per Share)")
        ax.legend()
        ax.grid(True)

        # Embed the Matplotlib figure into Tkinter window
        popup = tk.Toplevel()
        popup.title(f"{name} Share Price")

        canvas = FigureCanvasTkAgg(fig, master=popup)
        canvas.draw()
        canvas.get_tk_widget().pack()

        
if __name__ == "__main__":
    root = tk.Tk()
    app = CompanySelectionApp(root, df_companies)
    root.mainloop()
